# How to use Plottr with QCoDeS for live plotting?

Note: This notebook assumes that the user has a `conda` environment on their system with latest versions of `plottr` installed. If you do not have plottr installed, please use command `pip install plottr` for installing it into your environment.

Let's begin!

First, make necessary imports.

In [1]:
import os
import numpy as np

import qcodes as qc
from qcodes import Station, initialise_or_create_database_at, \
    load_or_create_experiment, Measurement
from qcodes.tests.instrument_mocks import DummyInstrument, \
    DummyInstrumentWithMeasurement

qc.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\v-singak\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\v-singak\.qcodes\logs\201203-3532-qcodes.log
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\v-singak\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\v-singak\.qcodes\logs\201203-3532-qcodes.log


For this notebook, we create a mock station setup. 

In [2]:
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrumentWithMeasurement(name='dmm', setter_instr=dac)

station = qc.Station(dmm, dac)

After that, database needs to be initialized or created if it doesn't exist using `initialize_or_create_database` method. Furthermore, datasets are associated with experiments and by default the run is appended to the latest existing experiment. We can load or create an experiment using `load_or_create_experiment` function. 

In [3]:
db_file_path = os.path.join(os.getcwd(), 'plottr_for_live_plotting_tutorial.db')
initialise_or_create_database_at(db_file_path)
exp = load_or_create_experiment(experiment_name='plottr_for_live_plotting_with_subsecond_refresh_rate',
                          sample_name="no sample")

Now open an anaconda prompt and activate your conda environment where plottr is installed. Start Inspectr GUI with `plottr-inspectr` command. You will see a window as show below. In this window, set `Refresh interval (s)` to the desired value and check `Auto-plot new`. Using `File` button load the database initialized (or created) above.

<div>
<img src="files/plottr.jpg" width="400" height="400"/>
</div>

Start the measurement as shown below. Notice `meas.write_period = 0.1` . This sets the measurement write period to 0.1s and is recommended to be used with sub-second refresh interval for plottr.

<div>
<img src="files/plottr_set_refresh_interval_and_autoplot.jpg"/>
</div>

In [4]:

meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))

meas.write_period = 0.1


with meas.run() as datasaver:

    for set_v in np.linspace(0, 25, 10):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))
        
    dataset = datasaver.dataset


Starting experimental run with id: 1. 


On starting this measurement, a plot window (as shown below) will open automatically. This plot will keep refreshing at the interval rate set by you till the measurement runs.

<div>
<img src="files/plotWindow.jpg" width="500" height="600"/>
</div>